In [1]:
import cbor2
import sqlite3

In [2]:
db = sqlite3.connect("blueprints.db")
cursor = db.cursor()

In [3]:
cursor.execute("select type_id from material;")
type_id_list = list(map(lambda x: x[0], cursor.fetchall()))

In [5]:
cursor.execute("""create table if not exists material_recursive (
    type_id integer primary key not null unique,
    flag integer not null default 0,
    material blob not null,
    product blob not null,
    time integer not null default 1
);""")
db.commit()

In [4]:
cursor.execute("""create table if not exists product (
    product_id integer,
    amount integer,
    blp_id integer not null unique primary key
);""")
db.commit()

In [5]:
def execute_one(*args, **kwargs):
    cursor.execute(*args, **kwargs)
    return cursor.fetchone()

def execute_all(*args, **kwargs):
    cursor.execute(*args, **kwargs)
    return cursor.fetchall()

In [6]:
_select_flag_sql = "select flag from material where type_id == ?"

In [7]:
for type_id in type_id_list:
    blp_flag = execute_one(_select_flag_sql, (type_id,))
    if blp_flag[0] == 0:
        cursor.execute("insert into product values (null, null, ?)", (type_id,))
        continue
    product = execute_one("select product from material where type_id == ?;", (type_id,))
    if product:
        product = cbor2.loads(product[0])
        if product:
            for k, v in product.items():
                cursor.execute("insert into product values (?, ?, ?);", (k, v, type_id))
            continue
    cursor.execute("insert into product values (null, null, ?)", (type_id,))
db.commit()